In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# reading csv into dataframe
df = pd.read_excel('Model Database for Share (2).xlsx')

In [4]:
# Performing initial operations on columns
bp = (
    df
    .rename(columns={'Unnamed: 0': 'Subject'})
    .astype({'Subject': 'category'})
    .dropna(subset=['Date of Visit'])
    .drop(columns=['Last 4', 'DOB'])
    .assign(Subject=lambda x: x['Subject'].ffill())
)
# Identifying and dropping the NaN-named column
nan_column = bp.columns[bp.columns.isna()]
if len(nan_column) > 0:
    bp.drop(columns=nan_column, inplace=True)

In [5]:
# Remove extra spaces and lowercase all letters of the column headers
bp.columns = [str(col).strip().lower().replace(' ', '_') for col in bp.columns]

In [6]:
bp.rename(columns={'medications_(gluc,_bp,_lipid,_statin)': 'medications'}, inplace=True)

In [7]:
# Checking initial dtypes, totals, and NaNs
bp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 688 entries, 0 to 1168
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   subject           688 non-null    category      
 1   date_of_visit     688 non-null    datetime64[ns]
 2   age               653 non-null    float64       
 3   sex               459 non-null    object        
 4   wt                682 non-null    float64       
 5   sbp               675 non-null    float64       
 6   dbp               675 non-null    object        
 7   bmi               640 non-null    object        
 8   obese             471 non-null    object        
 9   pre-dm            464 non-null    object        
 10  dm_t2             464 non-null    object        
 11  a1c               525 non-null    object        
 12  trig              483 non-null    float64       
 13  hdl               483 non-null    object        
 14  total_chol        483 non-null

In [8]:
# Counting unique subjects
bp['subject'].nunique()

470

In [9]:
# handling missing data / outliers / duplicates
# Dropping specified columns from the DataFrame
bpp = bp.drop(columns=['age', 'sex', 'able_to_adhere', 'improved_energy?', 'hungry?', 'waist','cgm'])

In [11]:
# Define a function to filter the min and max dates for each subject
def filter_min_max_dates(group):
    return group[group['date_of_visit'].isin([group['date_of_visit'].min(), group['date_of_visit'].max()])]

In [12]:
# Apply the function to each group
bpa = bpp.groupby('subject', group_keys=False, observed=True).apply(filter_min_max_dates)

In [16]:
bpa.head(20)

,subject,date_of_visit,wt,sbp,dbp,bmi,obese,pre-dm,dm_t2,a1c,trig,hdl,total_chol,ldl,medications
0,1.0,2023-03-27,264.0,129.0,78,39.07,YES,NO,YES,6.4,187.0,54,113.0,22.1,"Atorvastatin, Losartan, Metformin,"
3,1.0,2023-11-14,265.0,130.0,78,39.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,2023-03-28,233.0,130.0,89,31.67,YES,NO,YES,6.7,63.0,49,146.0,84.6,"Lisinopril, alogliptin, empagliflozin, hctz/tr..."
5,2.0,2023-06-27,210.0,139.0,83,NaN,NaN,NaN,NaN,5.3,NaN,NaN,NaN,NaN,"Lisinopril, amlodipine,"
7,3.0,2023-03-28,230.0,146.0,94,29.59,NO,NO,YES,9,337.0,33,221.0,120.3,"aloglip, atorvastatin, hctz/lisinopril, metfor..."
9,4.0,2023-03-29,197.0,129.0,80,30.02,YES,NO,YES,6.5,438.0,23,116.0,53,"semaglutide, atenolol, rosuvastatin, fenofibra..."
10,4.0,2023-10-16,169.0,NaN,NaN,NaN,NaN,NaN,NaN,6.4,230.0,28,99.0,25,NaN
12,5.0,2023-03-30,226.0,117.0,88,32.5,YES,YES,NO,5.7,150.0,46,207.0,130.9,atorvastatin
14,6.0,2023-03-31,269.0,116.0,87,38.7,YES,NO,YES,6.9,372.0,28,122.0,19,"metf, aloglip, atorv, hydralazine, spironolact..."
16,6.0,2023-09-05,237.0,127.0,80,34.12,NaN,NaN,NaN,5.5,NaN,NaN,NaN,NaN,stop alog


In [13]:
bpp.to_csv('bpz.csv', index=False, header=True)